## Basic sqoop import Command

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--target-dir sqoop_import_dir

1. Target-dir must not exist beforehand. If it exists, the command while fail with the same error. 
2. This command will create the directory while sqooping, then would put the table data inside this directory as part files. 
3. There would be no subfolder created inside target-dir with the table name, datafiles would be put directly inside target-dir.

If target-dir is not provided, sqoop will create a directory in the current working directory with the same name as table name.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user

## Controlling Parellelism

1. To leverage parallelism, we need to provide number of mappers in the import command. There would be that many parallel imports executed, as mentioned in the --num-mappers parameter. 
2. That many number of part files would get created as there are number of mappers mentioned in the sqoop import command

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--target-dir sqoop_import_dir \
--num-mappers 3


3. When performing parallel imports, Sqoop needs a criterion by which it can split the workload. Sqoop uses a splitting column to split the workload. By default, Sqoop will identify the primary key column (if present) in a table and use it as the splitting column. The low and high values for the splitting column are retrieved from the database, and the map tasks operate on evenly-sized components of the total range. For example, if you had a table with a primary key column of id whose minimum value was 0 and maximum value was 1000, and Sqoop was directed to use 4 tasks, Sqoop would run four processes which each execute SQL statements of the form ```SELECT * FROM sometable WHERE id >= lo AND id < hi```, with (lo, hi) set to (0, 250), (250, 500), (500, 750), and (750, 1001) in the different tasks. **(In the above sqoop, user_id is the primary key of the table 'user', which has been used as splitting key as can be seen from the sqoop log printed on console)**

4. If the actual values for the primary key are not uniformly distributed across its range, then this can result in unbalanced tasks. You should explicitly choose a different column with the --split-by argument. For example, --split-by employee_id. This is also applicable for sqooping from a table which does not have a primary key, hence split-by column has to be mentioned **(Below we are sqooping from a mysql table 'logs' which had the columns logid,logdate,jobname,stepname,status,error_message. None of these has been made primary key of the table. Hence we have to either mention a column with --split-by parameter, or perform a sequential import using '-m 1'. Else it will throw error -** *ERROR tool.ImportTool: Error during import: No primary key could be found for table logs. Please specify one with --split-by or perform a sequential import with '-m 1'.*

5. The split-by column has to be an integer column. If a textual column is passed, it would throw error - *ERROR tool.ImportTool: Encountered IOException running import job: java.io.IOException: Generating splits for a textual index column allowed only
 in case of "-Dorg.apache.sqoop.splitter.allow_text_splitter=true" property passed as a parameter* 

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table logs \
--target-dir sqoop_import_dir \
--num-mappers 3 \
--split-by logid


### Warehouse-dir parameter

**1. warehouse-dir may exist beforehand, if does not exist it will be created while sqooping**
2. During sqooping, a subfolder would be created inside the warehouse-dir with same name as the table name
3. Data would be put inside this subfolder as part files

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--warehouse-dir sqoop_import_dir


### Using Options Files to Pass Arguments

1. When using Sqoop, the command line options that do not change from invocation to invocation can be put in an options file for convenience. An options file is a text file where each line identifies an option in the order that it appears otherwise on the command line.
2. To specify an options file, simply create an options file in a convenient location and pass it to the command line via --options-file argument.

import.txt \
import \
--connect \
jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username \
sqoopuser \
--password \
NHkkP876rp





In [0]:
sqoop --options-file /home/dataenggdatascfreelance1247/import.txt --table user


### Secure way of supplying password to the database

You should save the password in a file on the users home directory with 400 permissions and specify the path to that file using the --password-file argument, and is the preferred method of entering credentials. Sqoop will then read the password from the file and pass it to the MapReduce cluster using secure means with out exposing the password in the job configuration. The file containing the password can either be on the Local FS or HDFS

password.txt \
NHkkP876rp



In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser 
--password-file /home/dataenggdatascfreelance1247/password.txt


### Selecting the Data to Import

By default, all columns within a table are selected for import. You can select 
a subset of columns and control their ordering by using the --columns argument


In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--columns name,age,country

You can append a WHERE clause to this with the --where argument. For example: --where "id > 400"

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--columns name,age,country \
--where "user_id > 2"

### Free-form Query Imports

1. Sqoop can also import the result set of an arbitrary SQL query. Instead of using the --table, --columns and --where arguments, you can specify a SQL statement with the --query argument.
2. When importing a free-form query, you must specify a destination directory with --target-dir.
3. When importing query results in parallel, you must specify --split-by.
4. Must provide \$CONDITIONS in the query irrespective of whether actually using a where clause in the query or not.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--query "SELECT * FROM user where \$CONDITIONS" \
--target-dir sqoop_import_dir \
--split-by user_id

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--query "SELECT u.user_id,u.name,u.age,us.salary FROM user u JOIN user_salary us ON u.user_id = us.user_id WHERE u.user_id in (1,2) AND \$CONDITIONS" \
--num-mappers 1 \
--target-dir sqoop_import_dir

## Incremental Imports

1. The following arguments control incremental imports: 
--check-column (col) : Specifies the column to be examined when determining which rows to import. (the column should not be of type CHAR/NCHAR/VARCHAR/VARNCHAR/ LONGVARCHAR/LONGNVARCHAR)\
--incremental (mode) : Specifies how Sqoop determines which rows are new. Legal values for mode include 'append' and 'lastmodified'.\
--last-value (value) : Specifies the maximum value of the check column from the previous import.

2. You should specify 'append' mode when importing a table where new rows are continually being added with increasing row id values. You specify the column containing the row id with --check-column. Sqoop imports rows where the check column has a value greater than the one specified with --last-value (last-value will be that value which has been last imported from the table).

3. You should use 'lastmodified' when rows of the source table may be updated, and each such update will set the value of a last-modified column to the current timestamp. Rows where the check column holds a timestamp more recent than the timestamp specified with --last-value are imported.

4. If --last-value is noe supplied while importing, it will import all the data of the table.

5. At the end of an incremental import, the value which should be specified as --last-value for a subsequent import is printed to the screen.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--check-column user_id \
--incremental append \
--last-value 5

## File Formats

1. Delimited text is the default import format. You can also specify it explicitly by using the --as-textfile argument, with delimiter characters between individual columns and rows. These delimiters may be commas, tabs, or other characters. 
2. As output line formatting arguments, we use --enclosed-by <char>, --escaped-by <char>, --fields-terminated-by <char>, --lines-terminated-by <char>


In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--fields-terminated-by ',' \
--lines-terminated-by '\n' \
--target-dir sqoop_import_dir

3. If unambiguous delimiters cannot be presented, then use enclosing and escaping characters. The combination of (optional) enclosing and escaping characters will allow unambiguous parsing of lines. For example, suppose one column of a dataset contained the following values:\
`Some string, with a comma. \
Another "string with quotes"`\
The following arguments would provide delimiters which can be unambiguously parsed:\
`sqoop import --fields-terminated-by , --escaped-by \\ --enclosed-by '\"' ...`\
The result of the above arguments applied to the above dataset would be:\
`"Some string, with a comma.","1","2","3"... \
"Another \"string with quotes\"","4","5","6"...`

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--escaped-by \\ \
--enclosed-by '\"' \
--fields-terminated-by ',' \
--lines-terminated-by '\n' \
--target-dir sqoop_import_dir

4. Delimited text is appropriate for most non-binary data types. It also readily supports further manipulation by other tools, such as Hive.
5. SequenceFiles are a binary format that store individual records in custom record-specific data types. This format supports exact storage of all data in binary representations, and is appropriate for storing binary data (for example, VARBINARY columns)
6. Reading from SequenceFiles is higher-performance than reading from text files, as records do not need to be parsed.
7. Avro data files are a compact, efficient binary format that provides interoperability with applications written in other programming languages. Avro also supports versioning, so that when, e.g., columns are added or removed from a table, previously imported data files can be processed along with new ones.
8. The parameters used are, --as-avrodatafile, --as-sequencefile, --as-textfile, --as-parquetfile

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--as-sequencefile \
--target-dir sqoop_import_dir

# Importing Data Into Hive

1. Sqoop’s import tool’s main function is to upload your data into files in HDFS. If you have a Hive metastore associated with your HDFS cluster, Sqoop can also import the data into Hive by generating and executing a CREATE TABLE statement to define the data’s layout in Hive.
2. If the Hive table already exists, you can specify the --hive-overwrite option to indicate that existing table in hive must be replaced. 
3. After your data is imported into HDFS, Sqoop will generate a Hive script containing a CREATE TABLE operation defining your columns using Hive’s types, and a LOAD DATA INPATH statement to move the data files into Hive’s warehouse directory.
4. Sqoop will pass the field and record delimiters through to Hive. If you do not set any delimiters and do use --hive-import, the field delimiter will be set to ^A and the record delimiter will be set to \n to be consistent with Hive’s defaults.
5. The table name used in Hive is, by default, the same as that of the source table. You can control the output table name with the --hive-table option.


In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--hive-import \
--hive-table user_hive


6. Hive can put data into partitions for more efficient query performance. You can tell a Sqoop job to import data for Hive into a particular partition by specifying the --hive-partition-key and --hive-partition-value arguments. The partition value must be a string. Please see the Hive documentation for more details on partitioning.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--hive-import \
--hive-table user_hive \
--hive-partition-key user_id \
--hive-partition-value 5


7. Hive will have problems using Sqoop-imported data if your database’s rows contain string fields that have Hive’s default row delimiters (\n and \r characters) or column delimiters (\01 characters) present in them. You can use the --hive-drop-import-delims option to drop those characters on import to give Hive-compatible text data. Alternatively, you can use the --hive-delims-replacement option to replace those characters with a user-defined string on import to give Hive-compatible text data.

In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--hive-import \
--hive-table user_hive \
--hive-drop-import-delims

# Importing Data Into HBase

1. Each row of the input table will be transformed into an HBase Put operation to a row of the output table.
2. The key for each row is taken from a column of the input. By default Sqoop will use the split-by column as the row key column. If that is not specified, it will try to identify the primary key column, if any, of the source table. You can manually specify the row key column with --hbase-row-key
3. Each output column will be placed in the same column family, which must be specified with --column-family
4. If the target table and column family do not exist, the Sqoop job will exit with an error. You should create the target table and column family before running an import
5. If you specify --hbase-create-table, Sqoop will create the target table and column family if they do not exist.


In [0]:
sqoop import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--hbase-table user_hbase \
--hbase-row-key user_id \
--column-family user_col_family \
--hbase-create-table

# Sqoop import-all-tables

1. Data from each table is stored in a separate directory in HDFS.
2. For the import-all-tables tool to be useful, the following conditions must be met:

  - Each table must have a single-column primary key or --autoreset-to-one-mapper option must be used.
  - You must intend to import all columns of each table.
  - You must not intend to use non-default splitting column, nor impose any conditions via a WHERE clause.

3. --table, --split-by, --columns, and --where arguments are invalid for sqoop-import-all-tables.
4. --target-dir option can not be used, need to use --warehouse-dir option.

In [0]:
sqoop import-all-tables \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--warehouse-dir sqoop_import_dir \
--autoreset-to-one-mapper

# Sqoop export

1. The target table of sqoop export must already exist in the database.
2. By default, all columns within a table are selected for export. You can select a subset of columns and control their ordering by using the --columns argument. This should include a comma-delimited list of columns to export. For example: --columns "col1,col2,col3". Note that columns that are not included in the --columns parameter need to have either defined default value or allow NULL values. Otherwise your database will reject the imported data which in turn will make Sqoop job fail.
3. Since Sqoop breaks down export process into multiple transactions, it is possible that a failed export job may result in partial data being committed to the database. This can further lead to subsequent jobs failing due to insert collisions in some cases, or lead to duplicated data in others. You can overcome this problem by specifying a staging table via the --staging-table option which acts as an auxiliary table that is used to stage exported data. The staged data is finally moved to the destination table in a single transaction.
4. In order to use the staging facility, you must create the staging table prior to running the export job. This table must be structurally identical to the target table. This table should either be empty before the export job runs, or the --clear-staging-table option must be specified. If the staging table contains data and the --clear-staging-table option is specified, Sqoop will delete all of the data before starting the export job.

In [0]:
sqoop export \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--export-dir sqoop_import_dir \
--table user_export

In [0]:
sqoop export \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--staging-table user_export_staging \
--clear-staging-table \
--table user_export \
--export-dir sqoop_import_dir



# Inserts vs. Updates

1. By default, sqoop-export appends new rows to a table; each input record is transformed into an INSERT statement that adds a row to the target database table. 
2. If you specify the --update-key argument, Sqoop will instead modify an existing dataset in the database. Each input record is treated as an UPDATE statement that modifies an existing row. The row a statement modifies is determined by the column name(s) specified with --update-key
3. If an UPDATE statement modifies no rows, this is not considered an error; the export will silently continue. 
4. Depending on the target database, you may also specify the --update-mode argument with allowinsert mode if you want to update rows if they exist in the database already or insert rows if they do not exist yet.

In [0]:
sqoop export \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--export-dir sqoop_import_dir \
--table user_export \
--update-key user_id \
--update-mode allowinsert

# Saved Sqoop Jobs

1. Imports and exports can be repeatedly performed by issuing the same command multiple times. Especially when using the incremental import capability, this is an expected scenario. Sqoop allows you to define saved jobs which make this process easier. A saved job records the configuration information required to execute a Sqoop command at a later time.
2. Saved jobs remember the parameters used to specify a job, so they can be re-executed by invoking the job by its handle. If a saved job is configured to perform an incremental import, state regarding the most recently imported rows is updated in the saved job to allow the job to continually import only the newest rows.


In [0]:
sqoop job --create myjob -- import \
--connect jdbc:mysql://cxln2.c.thelab-240901.internal/sqoopex \
--username sqoopuser \
--password NHkkP876rp \
--table user \
--target-dir sqoop_import_dir

In [0]:
sqoop job --list

In [0]:
sqoop job --show myjob

In [0]:
sqoop job --exec myjob

The exec action allows you to override arguments of the saved job by supplying them after a --. For example, if the database were changed to require a username, we could specify the username and password with:

In [0]:
sqoop job --exec myjob -- --username someuser -P